In [1]:
import pandas as pd
import numpy as np
import re   #regex

Im Folgenden werden die beiden csv Dateien eingelesen und über die 'category_id' zusammengejoint. Dieser Datensatz heißt data. Darüber hinaus soll ein zweiter Datensatz erstellt werden, in dem jedes Video nur einmal vertreten ist. Dieser Datensatz heißt data_once.

Inspiration für Projekt -> Kategorie (bspw. anhand der Tags) vorhersagen

### Erster Datensatz: data

In [2]:
data_vid = pd.read_csv('C:/Users/Nutzer/Desktop/Uni/Semester IV/Machine Learning/Projekt/DEvideos.csv')
data_cat = pd.read_csv('C:/Users/Nutzer/Desktop/Uni/Semester IV/Machine Learning/Projekt/DE_category_id.csv')

#Kurze Anmerkung zum trending_date da ich es safe vergessen werde: das Format ist yy.dd.mm - wer zum Henker 
#hat sich dieses Format ausgedacht

In [3]:
#Select the two needed columns
data_cat = data_cat[['items__id','items__snippet__title']]
#Rename the dataframe
data_cat.rename({'items__id':'category_id', 'items__snippet__title': 'category_name'}, axis = 1, inplace = True)
#join the dataframes
data = pd.merge(data_vid, data_cat, on = 'category_id')

In [4]:
data.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name
0,LgVi6y5QIjM,17.14.11,Sing zu Ende! | Gesangseinlagen vom Feinsten |...,inscope21,24,2017-11-13T17:08:49.000Z,"inscope21|""sing zu ende""|""gesangseinlagen""|""ge...",252786,35885,230,1539,https://i.ytimg.com/vi/LgVi6y5QIjM/default.jpg,False,False,False,Heute gibt es mal wieder ein neues Format... w...,Entertainment
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97190,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",Entertainment
2,ZJ9We4bjcg0,17.14.11,18 SONGS mit Kelly MissesVlog (Sing-off),rezo,24,2017-11-12T13:10:36.000Z,"kelly|""missesvlog""|""kelly song""|""bausa""|""bausa...",822213,100684,2467,10244,https://i.ytimg.com/vi/ZJ9We4bjcg0/default.jpg,False,False,False,18 Song Mashup über den (veränderten) Beat von...,Entertainment
3,2hu_evXPpMM,17.14.11,Dagi Bee wird Heiraten | Coldmirror bekommt Eh...,HerrNewstime,24,2017-11-12T16:33:18.000Z,"Dagi Bee|""Heiraten""|""Coldmirror""|""YouTube""|""Tr...",228574,11349,990,1049,https://i.ytimg.com/vi/2hu_evXPpMM/default.jpg,False,False,False,Dagi Bee wird Heiraten | Coldmirror bekommt Eh...,Entertainment
4,OKYUtHvgMhc,17.14.11,Die Welt in 30 Jahren - Sommer 2047 - VOLKAN,VOLKAN,24,2017-11-12T16:09:44.000Z,"Welt|""Natur""|""Umwelt""|""Klima""|""Klimawandel""",37877,1839,327,170,https://i.ytimg.com/vi/OKYUtHvgMhc/default.jpg,False,False,False,"Über die Zukunft nachzudenken macht Spaß, oder...",Entertainment


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40584 entries, 0 to 40583
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40584 non-null  object
 1   trending_date           40584 non-null  object
 2   title                   40584 non-null  object
 3   channel_title           40584 non-null  object
 4   category_id             40584 non-null  int64 
 5   publish_time            40584 non-null  object
 6   tags                    40584 non-null  object
 7   views                   40584 non-null  int64 
 8   likes                   40584 non-null  int64 
 9   dislikes                40584 non-null  int64 
 10  comment_count           40584 non-null  int64 
 11  thumbnail_link          40584 non-null  object
 12  comments_disabled       40584 non-null  bool  
 13  ratings_disabled        40584 non-null  bool  
 14  video_error_or_removed  40584 non-null  bool  
 15  de

### Zweiter Datensatz: data_once
Erkenntnis: die Daten aktualisieren sich mit jedem Tag, den sie in den Trends bleiben

In [6]:
list_cols = data.columns.values.tolist()
list_cols.remove('category_id')
list_cols.remove('thumbnail_link')
data_once = data.loc[:, list_cols]

Nun soll ein Dataframe erzeugt werden, der die video_id und die aktuellste trending_dates beinhaltet. Der Dataframe
wird dann mit dem Dataframe data_once gejoint.

In [9]:
df_latest_vid = data_once.groupby(['video_id']).agg(trending_date=('trending_date', 'max'))  

,video_id,trending_date,title,channel_title,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name
0,--7vNbh4UNA,18.14.04,"Responding to ALL The Outrage, Ridiculous H3H3...",Philip DeFranco,2018-04-13T19:00:00.000Z,"Elizabeth Hurley|""Instagram""|""Outrage""|""scanda...",1082647,52113,1284,10602,False,False,False,"Thanks for tuning in this week, ya Beautiful B...",News & Politics
1,--I0aj8MUdU,18.06.02,E diela shqiptare - Ka nje mesazh per ty! (04 ...,RTV KLAN,2018-02-04T13:19:38.000Z,"tv klan|""televizioni klan""|""klanhd""|""ardit gje...",139025,490,78,22,False,False,False,Facebook: https://www.facebook.com/tvklan Talk...,Entertainment
2,--MtKsH5oBY,18.31.05,صحفي بين سبورت يكشف تفاصيل و كواليس استقالة زي...,RedsTech,2018-05-31T12:40:46.000Z,"RedsTech|""bein sports""|""ريال مدريد""|""real madr...",225445,2156,227,562,False,False,False,I للتواصل معنا Ihttps://www.facebook.com/RedsT...,Sports
3,--moh3hSPpY,18.03.01,Te vreau langa mine! (02.01.2018) - Editie COM...,Kanal D Romania,2018-01-02T13:30:00.000Z,"te vreau langa mine|""kanal d""|""kanal d romania...",17050,100,32,74,False,False,False,Te vreau langa mine! (02.01.2018) - Editie COM...,Entertainment
4,--rE6D0_jAw,18.30.04,HÖR AUF DAS SCHLECHTE ZU SEHEN!,Biyon,2018-04-29T14:35:57.000Z,"Motivation|""Positivität""|""Erfolg""|""Selbstbewus...",5406,964,4,191,False,False,False,"▶︎ Wie du vielleicht mitbekommen hast, ist mei...",Entertainment


In [8]:
data_once = pd.merge(df_latest_vid, data_once ,  on = ['video_id','trending_date'])
data_once.drop(columns=['trending_date', 'latest_trending_date_x'])
data_once

KeyError: "['latest_trending_date_x'] not found in axis"

Testen des Dataframes data_once:

In [ ]:
data_test1 = data_once.groupby(['video_id'])['video_id'].count()
data_test1

In [ ]:
data_test2 = data_once[data_once.video_id == 'zzHW6gJG7W0']
data_test2

In [ ]:
data_test3 = data[data.video_id == 'zzHW6gJG7W0']
data_test3

In [ ]:
data_once.head()

Die Datensätze werden gespeichtert, sodass andere Notebooks auf die Datensätze zugreifen können.

In [ ]:
%store data
%store data_once